In [ ]:
import torch
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import numpy as np
from keras.utils import np_utils

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.utils import np_utils
import numpy as np 
	
tokenizer = Tokenizer()

def dataset_preparation(data):

    # basic cleanup
    corpus = data.lower().split("\n")

    # tokenization	
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    # create input sequences using list of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = np_utils.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words

def create_model(predictors, label, max_sequence_len, total_words):
    
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences = True))
    # model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
    print(model.summary())
    return model 

def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        # predicted = model.predict_classes(token_list, verbose=0)
        predict_x=model.predict(token_list, verbose=0) 
        predicted=np.argmax(predict_x,axis=1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text



data = open('./train_small.txt').read()

predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)


Epoch 1/100
3017/3017 [==============================] - 49s 15ms/step - loss: 6.4676 - accuracy: 0.0893
Epoch 2/100
3017/3017 [==============================] - 45s 15ms/step - loss: 5.9634 - accuracy: 0.1178
Epoch 3/100
3017/3017 [==============================] - 44s 15ms/step - loss: 5.6996 - accuracy: 0.1299
Epoch 4/100
3017/3017 [==============================] - 44s 15ms/step - loss: 5.4551 - accuracy: 0.1490
Epoch 5/100
3017/3017 [==============================] - 44s 15ms/step - loss: 5.2189 - accuracy: 0.1660
Epoch 6/100
3017/3017 [==============================] - 44s 14ms/step - loss: 5.0291 - accuracy: 0.1778
Epoch 7/100
3017/3017 [==============================] - 44s 15ms/step - loss: 4.8686 - accuracy: 0.1859
Epoch 8/100
3017/3017 [==============================] - 44s 15ms/step - loss: 4.7295 - accuracy: 0.1925
Epoch 9/100
3017/3017 [==============================] - 47s 16ms/step - loss: 4.6044 - accuracy: 0.1976
Epoch 10/100
3017/3017 [==============================]